In [6]:
# Standard library imports
import os
import requests

# Third party imports
import ee
import ee.mapclient
import folium
from google.oauth2 import service_account
from osgeo import gdal
from pyproj import Transformer

from dotenv import load_dotenv

#Specify the path to the service account key file (found in .env)
load_dotenv()
key_path = os.getenv('GEE_AUTH_KEY_PATH')

# Define the correct scope for Earth Engine
SCOPES = ['https://www.googleapis.com/auth/earthengine']

# Load the service account credentials
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=SCOPES
)

# Initialize Earth Engine
ee.Initialize(credentials=credentials)
print("Earth Engine initialized successfully!")

Earth Engine initialized successfully!


In [7]:
# Define the bounding box in epsg 6514
upper_left = [252500, 287500]
bottom_right = [256000, 278000]

buffer_amt = 1e4

upper_left_buffered = [upper_left[0] - buffer_amt, upper_left[1] + buffer_amt]
bottom_right_buffered = [bottom_right[0] + buffer_amt, bottom_right[1] - buffer_amt]

# Create a transformer object to convert from EPSG:6514 to EPSG:4326
transformer = Transformer.from_crs("EPSG:6514", "EPSG:4326", always_xy=True)

# Convert the coordinates
upper_left_latlon = transformer.transform(upper_left_buffered[0], upper_left_buffered[1])
bottom_right_latlon = transformer.transform(bottom_right_buffered[0], bottom_right_buffered[1])

# Print the results
print("Upper Left (LatLon):", upper_left_latlon)
print("Bottom Right (LatLon):", bottom_right_latlon)

# Define the bounding box in lat/lon (EPSG:4326)
bounding_box = ee.Geometry.Polygon([
    [upper_left_latlon,
     [bottom_right_latlon[0], upper_left_latlon[1]],  # Upper-right corner
     bottom_right_latlon,
     [upper_left_latlon[0], bottom_right_latlon[1]],  # Lower-left corner
     upper_left_latlon]  # Closing the polygon
], proj='EPSG:4326')

# Image collection path
path_to_asset = 'projects/rap-data-365417/assets/vegetation-cover-v3'

# Load the image collection
image_collection = ee.ImageCollection(path_to_asset)

available_years = [
    1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
    1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
    2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
    2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023
]

# Initialize woodyCover as None
woodyCover = None

# Process each year in the list
for year in available_years:
    # Filter images for the specific year
    yearly_image = image_collection.filter(
        ee.Filter.calendarRange(year, year, 'year')
    ).median().clip(bounding_box)

    # Calculate woody cover (tree + shrub cover) - Corrected band selection
    treeCover = yearly_image.select('TRE').rename(f'tree_cover_{year}')
    shrubCover = yearly_image.select('SHR').rename(f'shrub_cover_{year}')
    woodyCoverYear = treeCover.add(shrubCover).rename(f'woody_cover_{year}')

    # Add the yearly woody cover as a new band - Handling the first year differently
    if woodyCover is None:
        woodyCover = woodyCoverYear
    else:
        woodyCover = woodyCover.addBands(woodyCoverYear)

# Check the final woodyCover image bands
print("Final woodyCover bands:", woodyCover.bandNames().getInfo())

Upper Left (LatLon): (-114.19134988481733, 46.830904023642205)
Bottom Right (LatLon): (-113.86209345623392, 46.57803622659363)
Final woodyCover bands: ['woody_cover_1986', 'woody_cover_1987', 'woody_cover_1988', 'woody_cover_1989', 'woody_cover_1990', 'woody_cover_1991', 'woody_cover_1992', 'woody_cover_1993', 'woody_cover_1994', 'woody_cover_1995', 'woody_cover_1996', 'woody_cover_1997', 'woody_cover_1998', 'woody_cover_1999', 'woody_cover_2000', 'woody_cover_2001', 'woody_cover_2002', 'woody_cover_2003', 'woody_cover_2004', 'woody_cover_2005', 'woody_cover_2006', 'woody_cover_2007', 'woody_cover_2008', 'woody_cover_2009', 'woody_cover_2010', 'woody_cover_2011', 'woody_cover_2012', 'woody_cover_2013', 'woody_cover_2014', 'woody_cover_2015', 'woody_cover_2016', 'woody_cover_2017', 'woody_cover_2018', 'woody_cover_2019', 'woody_cover_2020', 'woody_cover_2021', 'woody_cover_2022', 'woody_cover_2023']


In [8]:

# Define visualization parameters
vis_params = {'min': 0, 'max': 100, 'palette': ['440154', '482878', '3E4A89', '31688E', '26828E', '1F9E89', '35B779', '6DCD59', 'B4DE2C', 'FDE725']}

# Create a folium map object.
centroid = bounding_box.centroid()
centroid_coords = centroid.coordinates().getInfo()
m = folium.Map(location=[centroid_coords[1], centroid_coords[0]], zoom_start=12)

# List of years to display
years_to_display = [1986, 2023]

for year in years_to_display:
    # Get the band name for the current year
    band_name = f'woody_cover_{year}'

    # Check if the band exists
    if band_name in woodyCover.bandNames().getInfo():
        # Select the woody cover band for the current year
        woodyCover_year = woodyCover.select(band_name)

        # Add the image layer to the map
        map_id_dict = woodyCover_year.getMapId(vis_params)
        folium.TileLayer(
            tiles=map_id_dict['tile_fetcher'].url_format,
            attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
            overlay=True,
            name=f'woody_cover_{year}',
        ).add_to(m)
    else:
        print(f"Band '{band_name}' not found in woodyCover.")

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map
display(m)

In [13]:
rap_dir = 'data/raster/rap'
os.makedirs(rap_dir, exist_ok=True)

# Set the GTIFF_SRS_SOURCE configuration option to EPSG
gdal.SetConfigOption('GTIFF_SRS_SOURCE', 'EPSG')

# Assuming 'woodyCover' is your ee.Image and 'bounding_box' is your ee.Geometry

for band_name in woodyCover.bandNames().getInfo():
    # Choose a value that doesn't exist in your data and doesn't have meaning in the context of your data.
    remapped_image = woodyCover.select(band_name).unmask(255)

    # Set an explicit "no data" value to null in the download URL parameters
    url = remapped_image.getDownloadURL({
        'scale': 30,
        'crs': 'EPSG:6514',
        'region': bounding_box,
        'format': 'GeoTIFF',
        'formatOptions': {
            'cloudOptimized': True,
            'noData': 255
        }
    })

    response = requests.get(url)
    
    # Create output file path
    output_file = os.path.join(rap_dir, f"{band_name}_temp.tif")
    
    # Write response content directly to final file
    with open(output_file, 'wb') as f:
        f.write(response.content)
    
    print(f"Downloaded file {output_file}")

    # Set nodata value to 255
    dataset = gdal.Open(output_file, gdal.GA_Update)
    band = dataset.GetRasterBand(1)
    band.SetNoDataValue(255)
    dataset = None

Downloaded file data/raster/rap/woody_cover_1986_temp.tif


Warning 1: TIFFWriteDirectorySec:Creating TIFF with legacy Deflate codec identifier, COMPRESSION_ADOBE_DEFLATE is more widely supported


Downloaded file data/raster/rap/woody_cover_1987_temp.tif


Warning 1: TIFFWriteDirectorySec:Creating TIFF with legacy Deflate codec identifier, COMPRESSION_ADOBE_DEFLATE is more widely supported


Downloaded file data/raster/rap/woody_cover_1988_temp.tif


...

Warning 1: TIFFWriteDirectorySec:Creating TIFF with legacy Deflate codec identifier, COMPRESSION_ADOBE_DEFLATE is more widely supported


Downloaded file data/raster/rap/woody_cover_2022_temp.tif


Warning 1: TIFFWriteDirectorySec:Creating TIFF with legacy Deflate codec identifier, COMPRESSION_ADOBE_DEFLATE is more widely supported


Downloaded file data/raster/rap/woody_cover_2023_temp.tif


Warning 1: TIFFWriteDirectorySec:Creating TIFF with legacy Deflate codec identifier, COMPRESSION_ADOBE_DEFLATE is more widely supported


In [14]:
for band_name in woodyCover.bandNames().getInfo():
    input_file = os.path.join(rap_dir, f"{band_name}_temp.tif")
    output_file = input_file.replace("_temp", "")
    # Use gdal_translate to crop the image in place
    gdal.Translate(output_file, input_file, projWin=[upper_left[0], upper_left[1], bottom_right[0], bottom_right[1]])
    
    # Remove original and rename temp file
    os.remove(input_file)
    
    print(f"Cropped file {input_file}")

Cropped file data/raster/rap/woody_cover_1986_temp.tif
Cropped file data/raster/rap/woody_cover_1987_temp.tif
Cropped file data/raster/rap/woody_cover_1988_temp.tif
Cropped file data/raster/rap/woody_cover_1989_temp.tif
Cropped file data/raster/rap/woody_cover_1990_temp.tif
Cropped file data/raster/rap/woody_cover_1991_temp.tif
Cropped file data/raster/rap/woody_cover_1992_temp.tif
Cropped file data/raster/rap/woody_cover_1993_temp.tif
Cropped file data/raster/rap/woody_cover_1994_temp.tif
Cropped file data/raster/rap/woody_cover_1995_temp.tif
Cropped file data/raster/rap/woody_cover_1996_temp.tif
Cropped file data/raster/rap/woody_cover_1997_temp.tif
Cropped file data/raster/rap/woody_cover_1998_temp.tif
Cropped file data/raster/rap/woody_cover_1999_temp.tif
Cropped file data/raster/rap/woody_cover_2000_temp.tif
Cropped file data/raster/rap/woody_cover_2001_temp.tif
Cropped file data/raster/rap/woody_cover_2002_temp.tif
Cropped file data/raster/rap/woody_cover_2003_temp.tif
Cropped fi